In [2]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular
from hist.axis import IntCategory
from hist.axis import StrCategory

from pathlib import Path

from typing import Optional, Union

from matplotlib.colors import Colormap, ListedColormap
from matplotlib.colors import LogNorm

In [3]:
from functools import cache
from functools import cached_property

@cache
def get_segment(ring: int, station: int, sector: int, subsector: int) -> int:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L361-L368
    """
    nsub = 3 if ring == 1 and station > 1 else 6
    return subsector + nsub * (sector - 1)


@cache
def get_roll_name(region: int, ring: int, station: int, sector: int, layer: int,
             subsector: int, roll: int
) -> str:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L11-L87
    """
    if region == 0:
        name = f'W{ring:+d}_RB{station}'

        if station <= 2:
            name += 'in' if layer == 1 else 'out'
        else:
            if sector == 4 and station == 4:
                name += ['--', '-', '+', '++'][subsector - 1]
            elif (station == 3) or (station == 4 and sector not in (4, 9, 11)):
                name += '-' if subsector == 1 else '+'
        name += f'_S{sector:0>2d}_'
        name += ['Backward', 'Middle', 'Forward'][roll - 1]
    else:
        segment = get_segment(ring, station, sector, subsector)
        name = f'RE{station * region:+d}_R{ring}_CH{segment:0>2d}_'
        name += ['A', 'B', 'C', 'D', 'E'][roll - 1]
    return name

In [4]:
def load_data(input_path: Path,
                       roll_blacklist_path: Optional[Path] = None,
)->dict:
    data = uproot.open(f"{str(input_path)}:tree").arrays(library='np')

    if roll_blacklist_path is None:
        roll_blacklist = set()
    else:
        with open(roll_blacklist_path) as stream:
            roll_blacklist = set(json.load(stream))

    fiducial_mask = data['is_fiducial']

    for key, values in data.items():
        data[key] = data[key][fiducial_mask]

    data['roll_name'] = np.array([get_roll_name(
                                                data['region'][idx], data['ring'][idx], data['station'][idx],
                                                data['sector'][idx], data['layer'][idx], data['subsector'][idx], data['roll'][idx]) 
                                                for idx in range(len(data['region']))])
                                            
    is_blacklist = np.vectorize(lambda item: item in roll_blacklist)
    blacklist_mask = is_blacklist(data['roll_name'])

    for key, values in data.items():
        data[key] = data[key][~blacklist_mask]

    return data

In [5]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')

data = load_data(
    input_path = working_dir / 'data' / 'Run2022.root',
    roll_blacklist_path = working_dir / 'blacklist' / 'roll-blacklist.json',
)

In [7]:
region = 'all'

if region == "all":
    is_region = np.vectorize(lambda item: type(item) is str)
elif region == "barrel":
    is_region = np.vectorize(lambda item: item.startswith('W'))
elif region == "endcap":
    is_region = np.vectorize(lambda item: item.startswith('RE'))

region_mask = is_region(data['roll_name'])

region_data = {}

for key, values in data.items():
    region_data[key] = data[key][region_mask]

runs = np.unique(region_data['run'])
print(runs)

for run in runs:
    run_data = {}
    run_data = region_data[region_data['run']]



[355374 355381 355429 355435 355442 355443 355444 355445 355454 355455
 355456 355558 355559 355679 355680 355768 355769 355862 355863 355870
 355871 355872 355892 355912 355913 355921 355933 355942 355988 355989
 355998 355999 356004 356005 356043 356071 356074 356075 356076 356077
 356135 356309 356316 356322 356323 356371 356375 356378 356381 356383
 356385 356386 356426 356428 356433 356434 356435 356446 356523 356531
 356563 356568 356569 356570 356576 356578 356580 356582 356614 356615
 356619 356810 356811 356812 356813 356814 356815 356824 356908 356919
 356937 356946 356947 356948 356949 356951 356954 356955 356956 356968
 356969 356970 356998 356999 357000 357001 357079 357080 357081 357101
 357102 357104 357106 357112 357268 357271 357328 357329 357330 357331
 357332 357333 357401 357406 357438 357440 357441 357442 357447 357472
 357478 357479 357482 357538 357542 357550 357610 357611 357612 357613
 357688 357696 357697 357698 357699 357700 357701 357705 357706 357720
 35773